In [1]:
import torch
from torch_scatter import scatter

In [32]:
x = torch.randn((9, 4))
cluster = torch.cat((torch.zeros(4), torch.ones(5))).long()
print(x)
print(cluster)

tensor([[ 0.5863,  1.5484, -0.5511,  0.0053],
        [ 0.0913, -0.1565, -0.6877, -1.3328],
        [-0.9014,  1.7190, -1.1770, -0.0879],
        [-1.5232,  0.1608, -0.6785, -0.0592],
        [-0.2632,  0.2822, -0.8354,  0.0620],
        [-0.1202,  0.2351, -1.9728, -0.2511],
        [-0.7063, -0.5570,  0.7382,  0.4196],
        [ 0.7178,  0.4264, -0.5496,  0.8273],
        [ 0.2326, -0.4034, -0.8150, -0.7342]])
tensor([0, 0, 0, 0, 1, 1, 1, 1, 1])


In [4]:
c = scatter(x, cluster, dim=0, reduce='max')
print(c)
print(c.shape)
# print(c[cluster])

tensor([[1.3134, 1.0530, 0.4749, 1.9412],
        [1.4040, 1.5390, 1.3179, 1.1116]])
torch.Size([2, 4])


In [16]:
# > pytorch 1.12

x = torch.randn((9, 4))

cluster = torch.cat((torch.zeros(4), torch.ones(5))).long().unsqueeze(1).repeat((1,4))

output = torch.zeros((2,4))
out = output.scatter_reduce(0, cluster, x, reduce="amax", include_self=False)

out = out[cluster[:,0]]

"""
In [1]: x
Out[1]: 
tensor([[-1.1713,  0.1483,  2.1248, -0.5578],
        [-0.6092, -1.7833, -0.2607, -0.2200],
        [ 1.8295, -1.1308, -0.2040, -1.4511],
        [ 1.4316, -1.1042,  0.8249, -0.6503],
        [ 1.0343, -1.4297,  1.3399, -0.2548],
        [ 0.6643,  0.1225,  0.8947,  1.6692],
        [ 0.8114,  0.9371, -0.6293,  0.6385],
        [ 0.0514,  0.2745, -0.0692,  0.5211],
        [ 0.0902, -0.4289,  0.2918,  0.4679]])
        
In [2]: cluster
Out[2]: 
tensor([[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [1, 1, 1, 1],
        [1, 1, 1, 1],
        [1, 1, 1, 1],
        [1, 1, 1, 1],
        [1, 1, 1, 1]])
        
        
In [3]: out[cluster[:,0]]
Out[3]: 
tensor([[ 1.8295,  0.1483,  2.1248, -0.2200],
        [ 1.8295,  0.1483,  2.1248, -0.2200],
        [ 1.8295,  0.1483,  2.1248, -0.2200],
        [ 1.8295,  0.1483,  2.1248, -0.2200],
        [ 1.0343,  0.9371,  1.3399,  1.6692],
        [ 1.0343,  0.9371,  1.3399,  1.6692],
        [ 1.0343,  0.9371,  1.3399,  1.6692],
        [ 1.0343,  0.9371,  1.3399,  1.6692],
        [ 1.0343,  0.9371,  1.3399,  1.6692]])

"""

AttributeError: 'Tensor' object has no attribute 'scatter_reduce'

In [63]:
cluster_expand = cluster.unsqueeze(-1).expand(x.shape)

print(cluster_expand.shape)
print(x)
print(cluster_expand)

torch.Size([9, 4])
tensor([[ 0.5863,  1.5484, -0.5511,  0.0053],
        [ 0.0913, -0.1565, -0.6877, -1.3328],
        [-0.9014,  1.7190, -1.1770, -0.0879],
        [-1.5232,  0.1608, -0.6785, -0.0592],
        [-0.2632,  0.2822, -0.8354,  0.0620],
        [-0.1202,  0.2351, -1.9728, -0.2511],
        [-0.7063, -0.5570,  0.7382,  0.4196],
        [ 0.7178,  0.4264, -0.5496,  0.8273],
        [ 0.2326, -0.4034, -0.8150, -0.7342]])
tensor([[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [1, 1, 1, 1],
        [1, 1, 1, 1],
        [1, 1, 1, 1],
        [1, 1, 1, 1],
        [1, 1, 1, 1]])


In [75]:
torch.max(x[cluster==torch.unique(cluster)[0]], dim=0)[0]

tensor([ 0.5863,  1.7190, -0.5511,  0.0053])

In [76]:
torch.max(x[cluster==torch.unique(cluster)[1]], dim=0)[0]

tensor([0.7178, 0.4264, 0.7382, 0.8273])

In [29]:
a = torch.tensor([[1,2,3,5], [4,4,5,8], [9,9,8,1]])
b = torch.tensor([2,0,0,1,1,1])
a[b]

tensor([[9, 9, 8, 1],
        [1, 2, 3, 5],
        [1, 2, 3, 5],
        [4, 4, 5, 8],
        [4, 4, 5, 8],
        [4, 4, 5, 8]])

In [54]:
a.numel()

12

In [109]:
src = torch.tensor([1, 3, 2, 4, 5, 6])
index = torch.tensor([0, 1, 0, 1, 1, 3])
dim = 0

out = scatter(src, index, dim=dim, reduce='max')
print(out)
print(out.shape)

tensor([2, 5, 0, 6])
torch.Size([4])


In [108]:
src = torch.tensor([
                    [[1, 1], 
                     [3, 3]], 
                    
                    [[2, 2], 
                     [4, 4]]
                    ])
index = torch.tensor([[0, 0], [0, 5]])
dim = 1

out = scatter(src, index, dim=dim, reduce='max')
print(out)
print(out.shape)

tensor([[[3, 3],
         [0, 0],
         [0, 0],
         [0, 0],
         [0, 0],
         [0, 0]],

        [[2, 2],
         [0, 0],
         [0, 0],
         [0, 0],
         [0, 0],
         [4, 4]]])
torch.Size([2, 6, 2])


In [103]:
torch.tensor([[[1, 1], [3, 3]], [[2, 2], [4, 4]]]).shape

torch.Size([2, 2, 2])